In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark").getOrCreate()

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=90c95f94c71b075f7d7448e3774f84baa2bd310812ae4cdc701c213132d8df73
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
#Read the state.txt file. Create RDD And convert to get codes
stateRDD=spark.sparkContext.textFile('state.txt')
stateKey=stateRDD.map(lambda word:(word.split(' ')[0],word.split(' ')[1]))
code_dict={}
for val in stateKey.collect():
  code_dict[val[0]]=val[1]
mapData=spark.sparkContext.broadCast(code_dict)

#create RDD for citizen details file
cRDD=spark.sparkContext.textFile('citizens.txt')


def getStateCode(state,codes):
  splitData=state.split(' ')
  splitData[4]=codes.value.get(splitData[4])
  newData=''
  newData=newData.join(splitData)
  return newData

mapCitizen=cRDD.map(lambda data: getStateCode(data,mapData)).collect()
mapCitizen.repartition(1).saveAsTextFile('/modified_citizens') #Saving the updated file in RDD